In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow import keras
from tensorflow.keras import layers
from keras import models, layers
import tqdm
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import KFold
import os
import os, shutil 
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [ ]:
path = '/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/'
data = pd.read_csv(path+'icml_face_data.csv')
emotions = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}
classes = dict(zip(range(0, 7), (((data[data[' Usage']=='Training']['emotion'].value_counts()).sort_index())/len(data[data[' Usage']=='Training']['emotion'])).tolist()))

In [ ]:
X = data.emotion
kf = KFold(n_splits=6)
for train, test in kf.split(X):
    print("%s %s" % (train, test))

In [ ]:
print(data)

In [ ]:
# Function to parse data into right format
# Output: Image in right shaped and normalized + labels
def parse_data(data):
    image_array = np.zeros(shape=(len(data), 48, 48)) # 1
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48)) # 1
        image_array[i] = image
        
    return image_array, image_label

# Splitting the data into train, validation and testing set thanks to Usage column
train_imgs, train_lbls = parse_data(data[data[" Usage"] == "Training"])
val_imgs, val_lbls = parse_data(data[data[" Usage"] == "PrivateTest"])
test_imgs, test_lbls = parse_data(data[data[" Usage"] == "PublicTest"])

In [ ]:
train_images = train_imgs.reshape((train_imgs.shape[0], 48, 48, 1))
train_images = train_images.astype('float32')/255
val_images = val_imgs.reshape((val_imgs.shape[0], 48, 48, 1))
val_images = val_images.astype('float32')/255
test_images = test_imgs.reshape((test_imgs.shape[0], 48, 48, 1))
test_images = test_images.astype('float32')/255

In [ ]:
print("train shape", np.shape(train_imgs))
print("validation shape", np.shape(val_imgs))
print("validatio shape", np.shape(val_imgs))

In [ ]:
print(train_imgs)

Image files generation

In [ ]:
# os.mkdir("/kaggle/working/imgs")
# data = np.array(train_imgs[:5])
# i = 0
# for px_map in data:
#     i = i + 1
#     px_map = np.reshape(px_map, (48, 48))
#     image = Image.fromarray(px_map)
#     image = image.convert('RGB')
#     image.save('/kaggle/working/imgs/'+str(i)+'.bmp')

https://scikit-learn.org/stable/modules/cross_validation.html

MLP model

In [ ]:
# Building a MLP model based on LeNet architecture 
model_mlp = keras.Sequential()
model_mlp.add(layers.Flatten(input_shape=(48, 48, 1)))
model_mlp.add(layers.Dense(units=120, activation='relu'))
model_mlp.add(layers.Dense(units=84, activation='relu'))
model_mlp.add(layers.Dense(units=7, activation = 'softmax'))
model_mlp.compile(loss=keras.losses.SparseCategoricalCrossentropy(), optimizer=keras.optimizers.Adam(lr=1e-3), metrics=['accuracy'])
model_mlp.summary()

In [ ]:
# Training the model, and validating
model_mlp.fit(train_imgs, train_lbls, 
          epochs=10, batch_size=32, 
          validation_data=(val_imgs, val_lbls), verbose=1)

In [ ]:
from keras.utils import to_categorical
train_labels = to_categorical(train_lbls)
val_labels = to_categorical(val_lbls)
test_labels = to_categorical(test_lbls)

CNN model

In [ ]:
model_cnn = models.Sequential()
model_cnn.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(64, activation='relu'))
model_cnn.add(layers.Dense(7, activation='softmax'))

In [ ]:
model_cnn.compile(optimizer=keras.optimizers.Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
model_cnn.summary()

In [ ]:
history = model_cnn.fit(train_images, train_labels,
                    validation_data=(val_images, val_labels),
                    class_weight = classes,
                    epochs=100,
                    batch_size=32)

In [ ]:
#Train accuracy and validation accuracy vs epoch graph
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, label='Training acc')
plt.plot(epochs, val_acc, label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
test_prob = model_cnn.predict(test_images)
test_pred = np.argmax(test_prob, axis=1)
test_accuracy = np.mean(test_pred == test_lbls)

print(test_accuracy)

In [ ]:
conf_mat = confusion_matrix(test_lbls, test_pred)
pd.DataFrame(conf_mat, columns=emotions.values(), index=emotions.values())

1. model_cnn = models.Sequential()
model_cnn.add(layers.Conv2D(128, (3, 3), activation='elu', input_shape=(48, 48, 1)))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(128, (3, 3), activation='elu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='elu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='elu'))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(64, activation='selu'))
model_cnn.add(layers.Dense(7, activation='softmax')), optimizers.Adam(lr=1e-3), epochs=50, batch_size=32

<html>
 <head>
  <meta charset="utf-8">
 </head>
 <body>
  <table border="1">
   <caption>Nerural network results table</caption>
   <tr>
    <th></th>
    <th>Angry</th>   
    <th>Disgust</th>
    <th>Fear</th>
    <th>Happy</th>
    <th>Sad</th>
    <th>Surprise</th>
    <th>Neutral</th>
   </tr>
   <tr><td>Angry</td><td>205</td><td>2</td><td>60</td><td>49</td><td>81</td><td>14</td><td>56</td></tr>
   <tr><td>Disgust</td><td>24</td><td>1</td><td>12</td><td>6</td><td>6</td><td>0</td><td>7</td></tr>
   <tr><td>Fear</td><td>60</td><td>0</td><td>181</td><td>39</td><td>108</td><td>60</td><td>48</td></tr>
   <tr><td>Happy</td><td>45</td><td>1</td><td>28</td><td>695</td><td>48</td><td>18</td><td>60</td></tr>
   <tr><td>Sad</td><td>94</td><td>2</td><td>105</td><td>62</td><td>261</td><td>24</td><td>105</td></tr>
   <tr><td>Surprise</td><td>20</td><td>0</td><td>41</td><td>24</td><td>7</td><td>298</td><td>25</td></tr>
   <tr><td>Neutral</td><td>53</td><td>0</td><td>66</td><td>62</td><td>103</td><td>21</td><td>302</td></tr>
  </table>
 </body>
</html>

2. model_cnn = models.Sequential()
model_cnn.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(64, activation='selu'))
model_cnn.add(layers.Dense(7, activation='softmax')), optimizers.Adam(lr=1e-3), epochs=50, batch_size=512

<html>
 <head>
  <meta charset="utf-8">
 </head>
 <body>
  <table border="1">
   <caption>Nerural network results table</caption>
   <tr>
    <th></th>
    <th>Angry</th>   
    <th>Disgust</th>
    <th>Fear</th>
    <th>Happy</th>
    <th>Sad</th>
    <th>Surprise</th>
    <th>Neutral</th>
   </tr>
   <tr><td>Angry</td><td>142</td><td>0</td><td>71</td><td>45</td><td>144</td><td>8</td><td>57</td></tr>
   <tr><td>Disgust</td><td>18</td><td>0</td><td>14</td><td>5</td><td>18</td><td>0</td><td>1</td></tr>
   <tr><td>Fear</td><td>24</td><td>0</td><td>165</td><td>29</td><td>174</td><td>34</td><td>70</td></tr>
   <tr><td>Happy</td><td>19</td><td>0</td><td>33</td><td>701</td><td>74</td><td>11</td><td>57</td></tr>
   <tr><td>Sad</td><td>30</td><td>2</td><td>77</td><td>44</td><td>395</td><td>11</td><td>96</td></tr>
   <tr><td>Surprise</td><td>10</td><td>0</td><td>80</td><td>32</td><td>25</td><td>252</td><td>16</td></tr>
   <tr><td>Neutral</td><td>22</td><td>0</td><td>52</td><td>66</td><td>193</td><td>6</td><td>268</td></tr>
  </table>
 </body>
</html>

3. model_cnn = models.Sequential()
model_cnn.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(64, activation='selu'))
model_cnn.add(layers.Dense(7, activation='softmax')), optimizers.Adam(lr=1e-3), epochs=50, batch_size=32

<html>
 <head>
  <meta charset="utf-8">
 </head>
 <body>
  <table border="1">
   <caption>Nerural network results table</caption>
   <tr>
    <th></th>
    <th>Angry</th>   
    <th>Disgust</th>
    <th>Fear</th>
    <th>Happy</th>
    <th>Sad</th>
    <th>Surprise</th>
    <th>Neutral</th>
   </tr>
   <tr><td>Angry</td><td>186</td><td>0</td><td>93</td><td>40</td><td>77</td><td>23</td><td>48</td></tr>
   <tr><td>Disgust</td><td>18</td><td>1</td><td>11</td><td>8</td><td>9</td><td>1</td><td>8</td></tr>
   <tr><td>Fear</td><td>64</td><td>0</td><td>185</td><td>37</td><td>105</td><td>52</td><td>53</td></tr>
   <tr><td>Happy</td><td>32</td><td>0</td><td>45</td><td>695</td><td>39</td><td>29</td><td>55</td></tr>
   <tr><td>Sad</td><td>56</td><td>0</td><td>132</td><td>50</td><td>285</td><td>34</td><td>96</td></tr>
   <tr><td>Surprise</td><td>20</td><td>0</td><td>42</td><td>21</td><td>15</td><td>307</td><td>10</td></tr>
   <tr><td>Neutral</td><td>51</td><td>0</td><td>90</td><td>63</td><td>129</td><td>19</td><td>255</td></tr>
  </table>
 </body>
</html>

4. model_cnn = models.Sequential()
model_cnn.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(64, activation='selu'))
model_cnn.add(layers.Dense(7, activation='softmax')), optimizers.Adam(lr=1e-6), epochs=12, batch_size=512

<html>
 <head>
  <meta charset="utf-8">
 </head>
 <body>
  <table border="1">
   <caption>Nerural network results table</caption>
   <tr>
    <th></th>
    <th>Angry</th>   
    <th>Disgust</th>
    <th>Fear</th>
    <th>Happy</th>
    <th>Sad</th>
    <th>Surprise</th>
    <th>Neutral</th>
   </tr>
   <tr><td>Angry</td><td>242</td><td>0</td><td>49</td><td>30</td><td>73</td><td>12</td><td>61</td></tr>
   <tr><td>Disgust</td><td>30</td><td>0</td><td>6</td><td>4</td><td>9</td><td>1</td><td>6</td></tr>
   <tr><td>Fear</td><td>65</td><td>0</td><td>171</td><td>31</td><td>93</td><td>55</td><td>81</td></tr>
   <tr><td>Happy</td><td>66</td><td>0</td><td>36</td><td>672</td><td>33</td><td>17</td><td>71</td></tr>
   <tr><td>Sad</td><td>96</td><td>0</td><td>96</td><td>42</td><td>255</td><td>19</td><td>145</td></tr>
   <tr><td>Surprise</td><td>22</td><td>0</td><td>57</td><td>26</td><td>6</td><td>289</td><td>15</td></tr>
   <tr><td>Neutral</td><td>53</td><td>0</td><td>54</td><td>63</td><td>91</td><td>21</td><td>325</td></tr>
  </table>
 </body>
</html>

5. model_cnn = models.Sequential()
model_cnn.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(64, activation='selu'))
model_cnn.add(layers.Dense(7, activation='softmax')), optimizers.Adam(lr=1e-6), epochs=12, batch_size=512

<html>
 <head>
  <meta charset="utf-8">
 </head>
 <body>
  <table border="1">
   <caption>Nerural network results table</caption>
   <tr>
    <th></th>
    <th>Angry</th>   
    <th>Disgust</th>
    <th>Fear</th>
    <th>Happy</th>
    <th>Sad</th>
    <th>Surprise</th>
    <th>Neutral</th>
   </tr>
   <tr><td>Angry</td><td>0</td><td>0</td><td>0</td><td>467</td><td>0</td><td>0</td><td>0</td></tr>
   <tr><td>Disgust</td><td>0</td><td>0</td><td>0</td><td>56</td><td>0</td><td>0</td><td>0</td></tr>
   <tr><td>Fear</td><td>0</td><td>0</td><td>0</td><td>496</td><td>0</td><td>0</td><td>0</td></tr>
   <tr><td>Happy</td><td>0</td><td>0</td><td>0</td><td>895</td><td>0</td><td>0</td><td>0</td></tr>
   <tr><td>Sad</td><td>0</td><td>0</td><td>0</td><td>653</td><td>0</td><td>0</td><td>0</td></tr>
   <tr><td>Surprise</td><td>0</td><td>0</td><td>0</td><td>415</td><td>0</td><td>0</td><td>0</td></tr>
   <tr><td>Neutral</td><td>0</td><td>0</td><td>0</td><td>607</td><td>0</td><td>0</td><td>0</td></tr>
  </table>
 </body>
</html>

6. model_cnn = models.Sequential()
model_cnn.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(64, activation='selu'))
model_cnn.add(layers.Dense(7, activation='softmax')), optimizers.Adam(lr=1e-3), epochs=12, batch_size=512

<html>
 <head>
  <meta charset="utf-8">
 </head>
 <body>
  <table border="1">
   <caption>Nerural network results table</caption>
   <tr>
    <th></th>
    <th>Angry</th>   
    <th>Disgust</th>
    <th>Fear</th>
    <th>Happy</th>
    <th>Sad</th>
    <th>Surprise</th>
    <th>Neutral</th>
   </tr>
   <tr><td>Angry</td><td>111</td><td>0</td><td>36</td><td>96</td><td>118</td><td>18</td><td>88</td></tr>
   <tr><td>Disgust</td><td>15</td><td>0</td><td>7</td><td>9</td><td>14</td><td>0</td><td>11</td></tr>
   <tr><td>Fear</td><td>27</td><td>0</td><td>109</td><td>89</td><td>124</td><td>58</td><td>89</td></tr>
   <tr><td>Happy</td><td>22</td><td>0</td><td>21</td><td>724</td><td>50</td><td>17</td><td>61</td></tr>
   <tr><td>Sad</td><td>42</td><td>0</td><td>49</td><td>120</td><td>276</td><td>9</td><td>157</td></tr>
   <tr><td>Surprise</td><td>11</td><td>0</td><td>63</td><td>41</td><td>21</td><td>252</td><td>27</td></tr>
   <tr><td>Neutral</td><td>29</td><td>0</td><td>22</td><td>99</td><td>120</td><td>21</td><td>328</td></tr>
  </table>
 </body>
</html>

7. model_cnn = models.Sequential()
model_cnn.add(layers.Conv2D(128, (3, 3), activation='selu', input_shape=(48, 48, 1)))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(128, (3, 3), activation='selu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='selu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='selu'))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(64, activation='selu'))
model_cnn.add(layers.Dense(7, activation='softmax')), optimizers.Adam(lr=1e-3), epochs=12, batch_size=512

<html>
 <head>
  <meta charset="utf-8">
 </head>
 <body>
  <table border="1">
   <caption>Nerural network results table</caption>
   <tr>
    <th></th>
    <th>Angry</th>   
    <th>Disgust</th>
    <th>Fear</th>
    <th>Happy</th>
    <th>Sad</th>
    <th>Surprise</th>
    <th>Neutral</th>
   </tr>
   <tr><td>Angry</td><td>116</td><td>0</td><td>82</td><td>99</td><td>76</td><td>11</td><td>83</td></tr>
   <tr><td>Disgust</td><td>18</td><td>0</td><td>15</td><td>10</td><td>8</td><td>1</td><td>4</td></tr>
   <tr><td>Fear</td><td>17</td><td>0</td><td>188</td><td>91</td><td>69</td><td>48</td><td>83</td></tr>
   <tr><td>Happy</td><td>20</td><td>0</td><td>40</td><td>752</td><td>20</td><td>8</td><td>55</td></tr>
   <tr><td>Sad</td><td>38</td><td>0</td><td>102</td><td>143</td><td>193</td><td>8</td><td>169</td></tr>
   <tr><td>Surprise</td><td>8</td><td>0</td><td>61</td><td>45</td><td>10</td><td>250</td><td>41</td></tr>
   <tr><td>Neutral</td><td>21</td><td>0</td><td>48</td><td>113</td><td>67</td><td>18</td><td>340</td></tr>
  </table>
 </body>
</html>

8. model_cnn = models.Sequential()
model_cnn.add(layers.Conv2D(128, (3, 3), activation='elu', input_shape=(48, 48, 1)))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(128, (3, 3), activation='elu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='elu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='elu'))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(64, activation='selu'))
model_cnn.add(layers.Dense(7, activation='softmax')), optimizers.Adam(lr=1e-3), epochs=100, batch_size=32

<html>
 <head>
  <meta charset="utf-8">
 </head>
 <body>
  <table border="1">
   <caption>Nerural network results table</caption>
   <tr>
    <th></th>
    <th>Angry</th>   
    <th>Disgust</th>
    <th>Fear</th>
    <th>Happy</th>
    <th>Sad</th>
    <th>Surprise</th>
    <th>Neutral</th>
   </tr>
   <tr><td>Angry</td><td>191</td><td>2</td><td>55</td><td>55</td><td>79</td><td>14</td><td>56</td></tr>
   <tr><td>Disgust</td><td>14</td><td>8</td><td>11</td><td>2</td><td>12</td><td>1</td><td>7</td></tr>
   <tr><td>Fear</td><td>57</td><td>3</td><td>168</td><td>48</td><td>87</td><td>41</td><td>48</td></tr>
   <tr><td>Happy</td><td>37</td><td>0</td><td>24</td><td>702</td><td>39</td><td>21</td><td>60</td></tr>
   <tr><td>Sad</td><td>73</td><td>4</td><td>87</td><td>69</td><td>269</td><td>11</td><td>105</td></tr>
   <tr><td>Surprise</td><td>21</td><td>0</td><td>33</td><td>37</td><td>11</td><td>292</td><td>21</td></tr>
   <tr><td>Neutral</td><td>52</td><td>0</td><td>55</td><td>79</td><td>91</td><td>12</td><td>318</td></tr>
  </table>
 </body>
</html>

9. model_cnn = models.Sequential()
model_cnn.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(layers.MaxPool2D((2, 2)))
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(64, activation='selu'))
model_cnn.add(layers.Dense(7, activation='softmax')), optimizers.Adam(lr=1e-3), epochs=100, batch_size=32

<html>
 <head>
  <meta charset="utf-8">
 </head>
 <body>
  <table border="1">
   <caption>Nerural network results table</caption>
   <tr>
    <th></th>
    <th>Angry</th>   
    <th>Disgust</th>
    <th>Fear</th>
    <th>Happy</th>
    <th>Sad</th>
    <th>Surprise</th>
    <th>Neutral</th>
   </tr>
   <tr><td>Angry</td><td>199</td><td>1</td><td>71</td><td>45</td><td>77</td><td>21</td><td>53</td></tr>
   <tr><td>Disgust</td><td>22</td><td>0</td><td>9</td><td>6</td><td>9</td><td>0</td><td>10</td></tr>
   <tr><td>Fear</td><td>53</td><td>1</td><td>176</td><td>46</td><td>105</td><td>48</td><td>67</td></tr>
   <tr><td>Happy</td><td>43</td><td>1</td><td>42</td><td>686</td><td>41</td><td>19</td><td>63</td></tr>
   <tr><td>Sad</td><td>84</td><td>0</td><td>96</td><td>82</td><td>248</td><td>16</td><td>127</td></tr>
   <tr><td>Surprise</td><td>19</td><td>0</td><td>55</td><td>32</td><td>13</td><td>271</td><td>25</td></tr>
   <tr><td>Neutral</td><td>67</td><td>0</td><td>61</td><td>75</td><td>106</td><td>5</td><td>293</td></tr>
  </table>
 </body>
</html>

Cross-validation: when the n_splits parameter changes, the dimension of the matrix changes. In our case, it is 6, ie we have 6 samples to which our dataset is divided. We consider 6 emotions because so the sample quality assessment of the models will be equivalent. <br>
Convolutional neural network: when changing the number of layers almost does not affect the result, but when increasing the number of neurons in the layers to 128 in some functions ZNM does not work, changing the types of activation functions: using sigmoid data is very confused and the final loss because we do not have a binary amount of data. <br>

1. What is the purpose of different types of layers of convolutional neural networks? <br>
The convolutional layer is the main building block of ZNM. <br> The aggregation layer serves to gradually reduce the spatial size of the representation to reduce the number of parameters and computational volume in the network, and thus also to control retraining. <br> The truncated linear nodes layer solutions and networks in general, without affecting the receptive fields of the convolutional layer. <br> Neurons in the fully connected layer have connections with all the excitations of the previous layer, as can be seen in conventional neural networks. Their excitation can therefore be calculated by matrix multiplication, followed by a bias shift. <br> The loss layer defines how training penalizes the deviation between predicted and true labels, and is usually the final layer. It can use different loss functions (usually softmax and sigmoid) for different tasks. <br>
2. What are the main stages of a typical machine learning project? <br>
Business analysis, data analysis and preparation, modeling, solution evaluation, implementation.